# IBM Certificate: Capstone Project

_This notebook contains the code for the Capstone project (IBM Certificate)_

## 1.Segmenting and Clustering Neighborhoods in Toronto

We import the below libraries:
- Pandas
- Numpy
- Requests


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests

The data we want are the **postal codes** in Canada and in particular the ones corresponding to **Toronto** in the province of Ontario. We can find these data on Wikipedia. The aim is then to scrap the data on the webpage. 

This will be done using the library **_requests_**.

In [2]:
# URL of the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Extract the content
r = requests.get(url)

Let's check what r contains so we know how to retrieve, clean, organise the data.

In [3]:
# Check the status code
print('Status code: ', r.status_code)
# Check the encoding
print('Encoding: ', r.encoding)
# Check the data type
print('Data type: ', type(r))
# Check the header
print('Headers: ', r.headers['content-type'])

Status code:  200
Encoding:  UTF-8
Data type:  <class 'requests.models.Response'>
Headers:  text/html; charset=UTF-8


We need to read an **html** content. Let's use _read_html_ from **Pandas**' library to retrieve the data.

In [23]:
raw_data = pd.read_html(r.text)

In [32]:
postal_codes = raw_data[0]

print('\n\n Dimension of the dataframe: ', postal_codes.shape)
postal_codes.head()



 Dimension of the dataframe:  (180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We need to clean the data a bit:
- remove the rows with a borough _'Not assigned'_
- a borough that has no corresponding neighborhood will have as neighborhood the same name as the borough

In [29]:
filter_1 = (postal_codes['Borough'] != 'Not assigned')
print('{} rows have a borough not assigned and need to be removed.'.format(len(postal_codes)-filter_1[filter_1 == True].count()))
postal_codes = postal_codes[filter_1]

77 rows have a borough not assigned and need to be removed.


In [30]:
filter_2 = (postal_codes['Neighborhood'] != 'Not assigned' )
print('{} boroughs have no corresponding neighborhood.'.format(filter_2[filter_2 == True].count()))
postal_codes = postal_codes[filter_2]

103 boroughs have no corresponding neighborhood.


In [31]:
postal_codes.shape

(103, 3)